# Part 2 - Data Preprocessing and Modeling

This notebook will focus on preprocessing data to bring them optimal size and format using data augmentation parameters. Once the preprocessing complete, I will train various Tensorflow Keras "sequential" and ""convolutional" networks as well as pre-trained models to sample transfer learning to come up with best results. As the classes are imbalanced, my success metrics should be Precision and Recall. Specifically Recall score is the most important as our goal is to focus on minimizing false negative rates to not classify a patient as healthy while in fact they have pneumonia.

In [20]:
# Import libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from glob import glob

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import tensorflow as tf

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, ZeroPadding2D, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For reproducibility
np.random.seed(42)

In [21]:
train_dir = './data/chest_xray/train/'
test_dir = './data/chest_xray/test/'

In [22]:
classes = ['normal', 'pneumonia']

I manually checked the images and found that there are a lot of variations for such a small dataset. The hight/width ratio, zooming range, angle of the body etc features differ among differen Xray images. Even the physical dimensions of images are vastly different. This makes it harder to train a model that will give high accuracy rate. I decided to use generator class to generate more images within train data with optimal rotation_range, shear_range, zoom_range, horizontal_flip (mirroring randomly selected images) to get additional observations to train the model with.

In [23]:
# Augmentation configuration to be used while training
train_generator = ImageDataGenerator(
                            rotation_range=20,
                            width_shift_range=0.25,
                            height_shift_range=0.25,
                            rescale=1./255,
                            shear_range=0.25,
                            zoom_range=0.25,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )

I will resize images to 224x224 px value and turn them to grayscale to only save the brightness and get rid of RGB values as the images are alrady provided as grayscale. It will help the train process run faster.


In [24]:
train_set = DirectoryIterator(train_dir,
                             train_generator,
                             target_size = (224, 224),
                             color_mode = 'rgb',
                             batch_size = 16,
                             classes=classes,
                             class_mode = 'categorical')

Found 5232 images belonging to 2 classes.


In [25]:
# Augmentation configuration to be used for validation
test_generator = ImageDataGenerator(rescale=1./255)

In [26]:
test_set = DirectoryIterator(test_dir,
                             test_generator,
                             target_size = (224, 224),
                             color_mode = 'rgb',
                             batch_size = 16, # set batch size a number that divides sample size
                             classes=classes,
                             class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [27]:
# Tensor size of train images
train_set.image_shape

(224, 224, 3)

In [28]:
# Tensor size of test images
test_set.image_shape

(224, 224, 3)

In [29]:
train_size = len(train_set.filenames)
test_size = len(test_set.filenames)

In [30]:
train_size, test_size

(5232, 624)

## Model #1 Dense

In [ ]:
# Initialize the sequential model
model1 = Sequential()

# Add flatten layer as input layer
model1.add(Flatten(input_shape = train_set.image_shape))

# Add a densely-connected layer with X neurons
model1.add(Dense(units = 10000,
                 activation='relu'))

# Add regularization
model1.add(Dropout(rate = 0.30))

# Add a second densely-connected layer with X neurons
model1.add(Dense(units = 128,
                activation='relu'))

# Add a third densely-connected layer with X neurons
model1.add(Dense(units = 64,
                 activation='relu'))

# Add regularization
model1.add(Dropout(rate = 0.10))

# Add a fourth densely-connected layer with X neurons
model1.add(Dense(units = 32,
                 activation='relu'))

# Add output layer
model1.add(Dense(units = 2,
    activation='sigmoid'
))

In [ ]:
# Define optimizer
opt = SGD(lr=0.01)

In [ ]:
# Compile the first model
model1.compile(optimizer = opt,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Define early stopping
early_stopping_monitor = EarlyStopping(patience = 5, 
                                       monitor = "val_accuracy", 
                                       mode="max", 
                                       verbose = 2)

# Define batch size (a divisor of test sample size)
batch_size = 16

In [ ]:
# Fit model on training data
history = model1.fit_generator(generator = train_set,
                              validation_data = test_set,
                              epochs = 25,
#                               callbacks=[early_stopping_monitor],
                              steps_per_epoch = train_size/batch_size, 
                              validation_steps = test_size/batch_size,
                              shuffle=False,
                              verbose = 2)

In [ ]:
predictions = model1.predict_generator(generator = test_set, 
                                       verbose = 2,
                                       steps = test_size/batch_size
                                      )

In [ ]:
y_hat = np.argmax(predictions, axis = 1) 
y_true = test_set.classes

In [ ]:
len(y_true) == len(y_hat)

In [ ]:
y_hat

In [ ]:
y_true

In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
CM = confusion_matrix(y_true, y_hat)
fig, ax = plot_confusion_matrix(conf_mat=CM,  figsize=(5, 5))
plt.show()

## Model #2 CNN

In [ ]:
# Initialize the sequential model
model2 = Sequential()

# Add convolution and pooling as input layer
model2.add(Conv2D(filters = 32, # number of filters
                 kernel_size = (3, 3), # height/width of filter
                 input_shape = train_set.image_shape, # shape of input (image)
                 activation = 'relu')) # activation function

model2.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

# Add a second convolutional layer
model2.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3),
                 activation = 'relu'))

model2.add(MaxPooling2D(pool_size = (2, 2)))

# Add a flattening layer
model2.add(Flatten())

# Add a densely-connected layer
model2.add(Dense(units = 128,
                activation = 'relu'))

# Add output layer
model2.add(Dense(units = 2,
                activation = 'sigmoid'))

In [1]:
model2.summary()

NameError: name 'model2' is not defined

In [ ]:
model2.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Fit model on training data
history = model2.fit_generator(generator = train_set,
                              validation_data = test_set,
                              epochs = 10,
                              callbacks=[early_stopping_monitor],
                              steps_per_epoch = train_size/batch_size, 
                              validation_steps = test_size/batch_size,
                              shuffle = True,
                              verbose = 2)

## Model #3 - Pre-trained Model (VGG16 Convolutional Base)

I decided to use the artichecture of a pre-trained model as I was curious about transfer learning outcome. Transfer learning is using a pre-trained model and its weights on a different dataset. I chose to create an instance from VGG16 convolutional neural network model which is popular from ImageNet competition. This would allow me to save a lot of time while testing the performance of my data with an additional model.

In [12]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

Using TensorFlow backend.


In [13]:
# base_model = VGG16(weights = 'imagenet', 
#                  include_top = False,
#                  input_shape = train_set.image_shape)

In [14]:
model = Sequential()
model.add(Conv2D(input_shape=train_set.image_shape, filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="sigmoid"))

In [15]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [17]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='accuracy', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='max', save_freq=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [19]:
history = model.fit_generator(steps_per_epoch=100,
                              generator=train_set, 
                              validation_data=test_set, 
                              validation_steps=10,
                              epochs=100,
                              callbacks=[checkpoint,early])

Epoch 1/100

Epoch 00001: accuracy did not improve from 0.90625
  1/100 [..............................] - ETA: 19:29 - loss: 0.4895 - accuracy: 0.8125
Epoch 00001: accuracy did not improve from 0.90625
  2/100 [..............................] - ETA: 18:03 - loss: 0.5627 - accuracy: 0.7500
Epoch 00001: accuracy did not improve from 0.90625
  3/100 [..............................] - ETA: 17:53 - loss: 0.5385 - accuracy: 0.7708
Epoch 00001: accuracy did not improve from 0.90625
  4/100 [>.............................] - ETA: 17:53 - loss: 0.5626 - accuracy: 0.7500
Epoch 00001: accuracy did not improve from 0.90625
  5/100 [>.............................] - ETA: 17:51 - loss: 0.5626 - accuracy: 0.7500
Epoch 00001: accuracy did not improve from 0.90625
  6/100 [>.............................] - ETA: 18:09 - loss: 0.5745 - accuracy: 0.7396
Epoch 00001: accuracy did not improve from 0.90625
  7/100 [=>............................] - ETA: 18:28 - loss: 0.5525 - accuracy: 0.7589
Epoch 00001: a

 59/100 [================>.............] - ETA: 8:44 - loss: 0.5675 - accuracy: 0.7458
Epoch 00001: accuracy did not improve from 0.90625
 60/100 [=================>............] - ETA: 8:31 - loss: 0.5705 - accuracy: 0.7427
Epoch 00001: accuracy did not improve from 0.90625
 61/100 [=================>............] - ETA: 8:18 - loss: 0.5734 - accuracy: 0.7398
Epoch 00001: accuracy did not improve from 0.90625
 62/100 [=================>............] - ETA: 8:05 - loss: 0.5722 - accuracy: 0.7409
Epoch 00001: accuracy did not improve from 0.90625
 63/100 [=================>............] - ETA: 7:52 - loss: 0.5721 - accuracy: 0.7411
Epoch 00001: accuracy did not improve from 0.90625
 64/100 [==================>...........] - ETA: 7:39 - loss: 0.5739 - accuracy: 0.7393
Epoch 00001: accuracy did not improve from 0.90625
 65/100 [==================>...........] - ETA: 7:27 - loss: 0.5746 - accuracy: 0.7385
Epoch 00001: accuracy did not improve from 0.90625
 66/100 [==================>......

 17/100 [====>.........................] - ETA: 17:47 - loss: 0.5943 - accuracy: 0.7206
Epoch 00002: accuracy did not improve from 0.90625
 18/100 [====>.........................] - ETA: 17:34 - loss: 0.5927 - accuracy: 0.7222
Epoch 00002: accuracy did not improve from 0.90625
 19/100 [====>.........................] - ETA: 17:21 - loss: 0.5825 - accuracy: 0.7336
Epoch 00002: accuracy did not improve from 0.90625
 20/100 [=====>........................] - ETA: 17:07 - loss: 0.5760 - accuracy: 0.7406
Epoch 00002: accuracy did not improve from 0.90625
 21/100 [=====>........................] - ETA: 16:55 - loss: 0.5755 - accuracy: 0.7411
Epoch 00002: accuracy did not improve from 0.90625
 22/100 [=====>........................] - ETA: 16:41 - loss: 0.5777 - accuracy: 0.7386
Epoch 00002: accuracy did not improve from 0.90625
 23/100 [=====>........................] - ETA: 16:28 - loss: 0.5771 - accuracy: 0.7391
Epoch 00002: accuracy did not improve from 0.90625
 24/100 [======>...........

 76/100 [=====================>........] - ETA: 5:16 - loss: 0.5628 - accuracy: 0.7525
Epoch 00002: accuracy did not improve from 0.90625
 77/100 [======================>.......] - ETA: 5:02 - loss: 0.5618 - accuracy: 0.7532
Epoch 00002: accuracy did not improve from 0.90625
 78/100 [======================>.......] - ETA: 4:49 - loss: 0.5648 - accuracy: 0.7508
Epoch 00002: accuracy did not improve from 0.90625
 79/100 [======================>.......] - ETA: 4:36 - loss: 0.5658 - accuracy: 0.7500
Epoch 00002: accuracy did not improve from 0.90625
 80/100 [=======================>......] - ETA: 4:23 - loss: 0.5629 - accuracy: 0.7523
Epoch 00002: accuracy did not improve from 0.90625
 81/100 [=======================>......] - ETA: 4:10 - loss: 0.5647 - accuracy: 0.7508
Epoch 00002: accuracy did not improve from 0.90625
 82/100 [=======================>......] - ETA: 3:56 - loss: 0.5647 - accuracy: 0.7508
Epoch 00002: accuracy did not improve from 0.90625
 83/100 [=======================>.

 34/100 [=========>....................] - ETA: 15:25 - loss: 0.5769 - accuracy: 0.7371
Epoch 00003: accuracy did not improve from 0.90625
 35/100 [=========>....................] - ETA: 15:08 - loss: 0.5804 - accuracy: 0.7339
Epoch 00003: accuracy did not improve from 0.90625
 36/100 [=========>....................] - ETA: 14:53 - loss: 0.5799 - accuracy: 0.7344
Epoch 00003: accuracy did not improve from 0.90625
 37/100 [==========>...................] - ETA: 14:36 - loss: 0.5812 - accuracy: 0.7331
Epoch 00003: accuracy did not improve from 0.90625
 38/100 [==========>...................] - ETA: 14:21 - loss: 0.5843 - accuracy: 0.7303
Epoch 00003: accuracy did not improve from 0.90625
 39/100 [==========>...................] - ETA: 14:05 - loss: 0.5820 - accuracy: 0.7324
Epoch 00003: accuracy did not improve from 0.90625
 40/100 [===========>..................] - ETA: 13:50 - loss: 0.5782 - accuracy: 0.7359
Epoch 00003: accuracy did not improve from 0.90625
 41/100 [===========>......

...

 72/100 [====================>.........] - ETA: 8:25 - loss: 0.5609 - accuracy: 0.7535
Epoch 00016: accuracy did not improve from 0.93750
 73/100 [====================>.........] - ETA: 8:07 - loss: 0.5601 - accuracy: 0.7543
Epoch 00016: accuracy did not improve from 0.93750
 74/100 [=====================>........] - ETA: 7:49 - loss: 0.5601 - accuracy: 0.7542
Epoch 00016: accuracy did not improve from 0.93750
 75/100 [=====================>........] - ETA: 7:31 - loss: 0.5627 - accuracy: 0.7517
Epoch 00016: accuracy did not improve from 0.93750
 76/100 [=====================>........] - ETA: 7:12 - loss: 0.5618 - accuracy: 0.7525
Epoch 00016: accuracy did not improve from 0.93750
 77/100 [======================>.......] - ETA: 6:54 - loss: 0.5619 - accuracy: 0.7524
Epoch 00016: accuracy did not improve from 0.93750
 78/100 [======================>.......] - ETA: 6:36 - loss: 0.5619 - accuracy: 0.7524
Epoch 00016: accuracy did not improve from 0.93750
 79/100 [======================>..

 30/100 [========>.....................] - ETA: 20:26 - loss: 0.6033 - accuracy: 0.7104
Epoch 00017: accuracy did not improve from 0.93750
 31/100 [========>.....................] - ETA: 19:57 - loss: 0.6002 - accuracy: 0.7137
Epoch 00017: accuracy did not improve from 0.93750
 32/100 [========>.....................] - ETA: 19:29 - loss: 0.5991 - accuracy: 0.7148
Epoch 00017: accuracy did not improve from 0.93750
 33/100 [========>.....................] - ETA: 19:03 - loss: 0.5998 - accuracy: 0.7140
Epoch 00017: accuracy did not improve from 0.93750
 34/100 [=========>....................] - ETA: 18:36 - loss: 0.5954 - accuracy: 0.7188
Epoch 00017: accuracy did not improve from 0.93750
 35/100 [=========>....................] - ETA: 18:14 - loss: 0.5962 - accuracy: 0.7179
Epoch 00017: accuracy did not improve from 0.93750
 36/100 [=========>....................] - ETA: 17:50 - loss: 0.5937 - accuracy: 0.7205
Epoch 00017: accuracy did not improve from 0.93750
 37/100 [==========>.......

 89/100 [=========================>....] - ETA: 3:04 - loss: 0.5775 - accuracy: 0.7367
Epoch 00017: accuracy did not improve from 0.93750
 90/100 [==========================>...] - ETA: 2:47 - loss: 0.5780 - accuracy: 0.7361
Epoch 00017: accuracy did not improve from 0.93750
 91/100 [==========================>...] - ETA: 2:30 - loss: 0.5793 - accuracy: 0.7349
Epoch 00017: accuracy did not improve from 0.93750
 92/100 [==========================>...] - ETA: 2:13 - loss: 0.5812 - accuracy: 0.7330
Epoch 00017: accuracy did not improve from 0.93750
 93/100 [==========================>...] - ETA: 1:57 - loss: 0.5810 - accuracy: 0.7332
Epoch 00017: accuracy did not improve from 0.93750
 94/100 [===========================>..] - ETA: 1:40 - loss: 0.5815 - accuracy: 0.7327
Epoch 00017: accuracy did not improve from 0.93750
 95/100 [===========================>..] - ETA: 1:23 - loss: 0.5806 - accuracy: 0.7336
Epoch 00017: accuracy did not improve from 0.93750
 96/100 [=========================

 46/100 [============>.................] - ETA: 14:56 - loss: 0.5651 - accuracy: 0.7486
Epoch 00018: accuracy did not improve from 0.93750
 47/100 [=============>................] - ETA: 14:43 - loss: 0.5665 - accuracy: 0.7473
Epoch 00018: accuracy did not improve from 0.93750
 48/100 [=============>................] - ETA: 14:28 - loss: 0.5650 - accuracy: 0.7487
Epoch 00018: accuracy did not improve from 0.93750
 49/100 [=============>................] - ETA: 14:13 - loss: 0.5691 - accuracy: 0.7449
Epoch 00018: accuracy did not improve from 0.93750
 50/100 [==============>...............] - ETA: 13:56 - loss: 0.5716 - accuracy: 0.7425
Epoch 00018: accuracy did not improve from 0.93750
 51/100 [==============>...............] - ETA: 13:39 - loss: 0.5754 - accuracy: 0.7390
Epoch 00018: accuracy did not improve from 0.93750
 52/100 [==============>...............] - ETA: 13:21 - loss: 0.5739 - accuracy: 0.7404
Epoch 00018: accuracy did not improve from 0.93750
 53/100 [==============>...

KeyboardInterrupt: 